In [115]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/animeface007/gan.zip/21130.png
/kaggle/input/animeface007/gan.zip/9273.png
/kaggle/input/animeface007/gan.zip/18966.png
/kaggle/input/animeface007/gan.zip/14127.png
/kaggle/input/animeface007/gan.zip/18054.png
/kaggle/input/animeface007/gan.zip/9292.png
/kaggle/input/animeface007/gan.zip/11396.png
/kaggle/input/animeface007/gan.zip/14260.png
/kaggle/input/animeface007/gan.zip/4353.png
/kaggle/input/animeface007/gan.zip/7968.png
/kaggle/input/animeface007/gan.zip/15834.png
/kaggle/input/animeface007/gan.zip/6490.png
/kaggle/input/animeface007/gan.zip/5511.png
/kaggle/input/animeface007/gan.zip/6262.png
/kaggle/input/animeface007/gan.zip/2664.png
/kaggle/input/animeface007/gan.zip/12329.png
/kaggle/input/animeface007/gan.zip/17678.png
/kaggle/input/animeface007/gan.zip/8419.png
/kaggle/input/animeface007/gan.zip/9703.png
/kaggle/input/animeface007/gan.zip/13412.png
/kaggle/input/animeface007/gan.zip/14325.png
/kaggle/input/animeface007/gan.zip/18315.png
/kaggle/input/animef

In [116]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [117]:
random_seed = 42
torch.manual_seed(random_seed)

In [118]:
batch_size = 128
NUM_WORKERS = int(os.cpu_count() / 2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
latent_dim = 100
epochs = 50
OUTPUT_DIR = "./gan_generated_images"
os.makedirs(OUTPUT_DIR, exist_ok=True)
data_dir = '/kaggle/input/animeface007/gan.zip'

In [119]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),       
    transforms.ToTensor(),            
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])


In [120]:
dataset = ImageFolder(root=data_dir, transform=transform)
dataset_size = len(dataset)
print(dataset_size )

21551


In [121]:
train_size = int(0.9 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [122]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)

In [123]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.fc = nn.Linear(256 * 8 * 8, 1)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.conv2(x), 0.2)
        x = F.leaky_relu(self.conv3(x), 0.2)
        x = x.view(-1, 256 * 8 * 8)
        x = self.fc(x)
        return torch.sigmoid(x)

In [124]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 256 * 8 * 8)
        self.ct1 = nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1)
        self.ct2 = nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1)
        self.ct3 = nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1)

    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 256, 8, 8)
        x = F.relu(self.ct1(x))
        x = F.relu(self.ct2(x))
        x = torch.tanh(self.ct3(x))  # Output in range [-1, 1]
        return x


In [125]:
from torchvision.utils import save_image
def save_generated_images(generator, latent_dim, epoch, device):
    generator.eval()  # Set generator to evaluation mode
    with torch.no_grad():
        latent_space = torch.randn(16, latent_dim).to(device)  # Generate 16 random samples
        fake_images = generator(latent_space)
        fake_images = (fake_images + 1) / 2  # Rescale to [0, 1] for saving as images
        save_image(fake_images, os.path.join(OUTPUT_DIR, f"epoch_{epoch+1}.png"), nrow=4)
    generator.train()  # Reset generator to training mode

In [126]:
def train_gan(discriminator, generator, train_loader, latent_dim, epochs, device):
    discriminator.to(device)
    generator.to(device)
    criterion = nn.BCELoss()
    
    # Define the optimizers
    d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    # Learning rate schedulers
    scheduler_d = optim.lr_scheduler.StepLR(d_optimizer, step_size=10, gamma=0.5)  # Decay every 10 epochs
    scheduler_g = optim.lr_scheduler.StepLR(g_optimizer, step_size=10, gamma=0.5)  # Decay every 10 epochs

    for epoch in range(epochs):
        for real_images, _ in train_loader:
            real_images = real_images.to(device)

            # Train Discriminator
            d_optimizer.zero_grad()
            real_labels = torch.ones(real_images.size(0), 1).to(device)
            fake_labels = torch.zeros(real_images.size(0), 1).to(device)

            real_outputs = discriminator(real_images)
            d_real_loss = criterion(real_outputs, real_labels)

            latent_space = torch.randn(real_images.size(0), latent_dim).to(device)
            fake_images = generator(latent_space)
            fake_outputs = discriminator(fake_images.detach())
            d_fake_loss = criterion(fake_outputs, fake_labels)

            # Average the discriminator losses
            d_loss = (d_real_loss + d_fake_loss) / 2  # Division by 2
            d_loss.backward()
            d_optimizer.step()

            # Train Generator
            g_optimizer.zero_grad()
            fake_outputs = discriminator(fake_images)
            g_loss = criterion(fake_outputs, real_labels)
            g_loss.backward()
            g_optimizer.step()

        # Update the learning rate using the scheduler
        scheduler_d.step()
        scheduler_g.step()
        if (epoch + 1) % 5 == 0: 
            save_generated_images(generator, latent_dim, epoch, device)
            print(f"Epoch [{epoch+1}/{epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}")
            print(f"Discriminator learning rate: {scheduler_d.get_last_lr()[0]:.6f}, Generator learning rate: {scheduler_g.get_last_lr()[0]:.6f}")
            print(f"Images saved at epoch {epoch+1}.")
        else:
            print(f"Epoch [{epoch+1}/{epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}")

discriminator = Discriminator()
generator = Generator(latent_dim)

train_gan(discriminator, generator, train_loader, latent_dim, epochs=epochs, device=device)
    
     
    
     
     
     
          

Epoch [1/50], d_loss: 0.0815, g_loss: 3.6362
Epoch [2/50], d_loss: 0.3985, g_loss: 4.1997
Epoch [3/50], d_loss: 0.3991, g_loss: 2.4592
Epoch [4/50], d_loss: 0.3832, g_loss: 1.5352
Epoch [5/50], d_loss: 0.6765, g_loss: 1.6654
Discriminator learning rate: 0.000200, Generator learning rate: 0.000200
Images saved at epoch 5.
Epoch [6/50], d_loss: 0.3266, g_loss: 2.6305
Epoch [7/50], d_loss: 0.4450, g_loss: 1.5651
Epoch [8/50], d_loss: 0.4697, g_loss: 2.0047
Epoch [9/50], d_loss: 0.4780, g_loss: 1.5689
Epoch [10/50], d_loss: 0.6163, g_loss: 1.1752
Discriminator learning rate: 0.000100, Generator learning rate: 0.000100
Images saved at epoch 10.
Epoch [11/50], d_loss: 0.5537, g_loss: 0.6968
Epoch [12/50], d_loss: 0.5252, g_loss: 0.8898
Epoch [13/50], d_loss: 0.5174, g_loss: 1.4479
Epoch [14/50], d_loss: 0.5328, g_loss: 1.6963
Epoch [15/50], d_loss: 0.5549, g_loss: 0.9938
Discriminator learning rate: 0.000100, Generator learning rate: 0.000100
Images saved at epoch 15.
Epoch [16/50], d_loss: 